In [72]:
import pandas as pd
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import re
import numpy as np
import os


In [73]:
# ChromeDriver 시작 함수
def start_chromedriver():
    service = Service("/opt/homebrew/bin/chromedriver")  # chromedriver 경로 수정 필요
    driver = webdriver.Chrome(service=service)
    print("Chrome 드라이버 실행.")
    return service, driver

# ChromeDriver 종료 함수
def end_chromedriver(driver, service):
    driver.quit()
    print("Chrome 드라이버를 닫음.")
    if service:
        service.stop()
        print("ChromeDriver 서비스 종료.")

In [74]:
def get_urls():
    # 무신사 랭킹 페이지 URL
    ranking_urls = {
        "clothes_top": "https://www.musinsa.com/main/musinsa/ranking?skip_bf=Y&storeCode=musinsa&sectionId=200&categoryCode=001000",
        "outers": "https://www.musinsa.com/main/musinsa/ranking?skip_bf=Y&storeCode=musinsa&sectionId=200&categoryCode=002000",
        "pants": "https://www.musinsa.com/main/musinsa/ranking?skip_bf=Y&storeCode=musinsa&sectionId=200&categoryCode=003000",
        "bags": "https://www.musinsa.com/main/musinsa/ranking?skip_bf=Y&storeCode=musinsa&sectionId=200&categoryCode=004000",
        "skirts": "https://www.musinsa.com/main/musinsa/ranking?skip_bf=Y&storeCode=musinsa&sectionId=200&categoryCode=100000",
        "fashion_accessories": "https://www.musinsa.com/main/musinsa/ranking?skip_bf=Y&storeCode=musinsa&sectionId=200&categoryCode=101000",
        "digital_life": "https://www.musinsa.com/main/musinsa/ranking?skip_bf=Y&storeCode=musinsa&sectionId=200&categoryCode=102000",
        "shoes": "https://www.musinsa.com/main/musinsa/ranking?skip_bf=Y&storeCode=musinsa&sectionId=200&categoryCode=103000",
        "beauty_items": "https://www.musinsa.com/main/musinsa/ranking?skip_bf=Y&storeCode=musinsa&sectionId=200&categoryCode=104000",
        "sportswears": "https://www.musinsa.com/main/musinsa/ranking?skip_bf=Y&storeCode=musinsa&sectionId=200&categoryCode=017000",
        "underwears": "https://www.musinsa.com/main/musinsa/ranking?skip_bf=Y&storeCode=musinsa&sectionId=200&categoryCode=026000",
        "kids": "https://www.musinsa.com/main/musinsa/ranking?skip_bf=Y&storeCode=musinsa&sectionId=200&categoryCode=106000"
    }
    return ranking_urls

In [92]:
def extract_product_details(driver, product_id, date):
    try:
        # 상품 상세 페이지로 이동
        product_url = f"https://www.musinsa.com/products/{product_id}"
        print(f"접근 중: {product_url}")
        driver.get(product_url)
        time.sleep(3)  # 페이지 로드 대기


        # HTML 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')

        # 조회수 추출
        views = None
        views_label = soup.find("span", string="조회수")  # `text`를 `string`으로 변경
        if views_label:
            views_element = views_label.find_next("span")
            if views_element:
                views = views_element.text.strip()
                
        # 총 판매량 추출
        total_sales = None
        total_sales_label = soup.find("span", string="누적판매")  # `text`를 `string`으로 변경
        if total_sales_label:
            total_sales_element = total_sales_label.find_next("span")
            if total_sales_element:
                total_sales = total_sales_element.text.strip()

        
        # 데이터 추출
        likes_element = soup.select_one("span.text-xs.font-medium.font-pretendard")
        likes = likes_element.text.strip() if likes_element else None
        rating_element = soup.select_one("span.font-medium.text-black")
        rating = rating_element.text.strip() if rating_element else None

        if rating and not rating.replace(".", "", 1).isdigit():  # 소수점 허용, 숫자가 아닌 경우
            rating = None
        
            
        # 텍스트를 숫자로 변환
        rate = extract_currently_value(rating) if rating else None
        views = extract_currently_value(views) if views else None
        total_sales = extract_currently_value(total_sales) if total_sales else None
        likes = extract_currently_value(likes) if likes else None


        
        # 반환
        details = {
            "ProductID": product_id,
            "Date": date,
            "Views": views,  # 중복 제거
            "Likes": likes,  # 중복 제거
            "Rating": rating,  # 중복 제거
            "TotalSales": total_sales,  # 중복 제거
        }
        print(f"추출된 데이터: {details}")
        return details
    except Exception as e:
        print(f"상품 세부 정보 추출 오류 (ID: {product_id}): {e}")
        return None


In [76]:
def extract_currently_value(text):
    """'22만 회 이상' 형태의 텍스트를 숫자로 변환."""

    if isinstance(text, float) and np.isnan(text):  # NaN 값 처리
        return 0

    if isinstance(text, list):  # 리스트 처리
        text = text[0] if text else ""

    if not isinstance(text, str):  # 문자열이 아닌 경우 처리
        text = str(text)
    text = text.replace(" ", "") 
    match = re.search(r"\d+(\.\d+)?", text)

    if not match:  # 숫자가 없을 경우
        return np.nan
    number = float(match.group())
    
    if "백" in text:  # "백" 단위 처리
        number *= 100

    if "천" in text:  # "천" 단위 처리
        number *= 1000

    if "만" in text:  # "만" 단위 처리
        number *= 10000
    
    return int(number)

In [77]:
def process_detailed_data(driver, input_dir, output_dir):
    # 출력 디렉토리 생성
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # 입력 디렉토리 탐색 (폴더 및 파일 모두 처리 가능)
    for item in os.listdir(input_dir):
        item_path = os.path.join(input_dir, item)
        
        if os.path.isdir(item_path):  # 디렉토리인 경우 처리
            for csv_file in os.listdir(item_path):
                if csv_file.endswith(".csv"):  # CSV 파일만 처리
                    input_csv = os.path.join(item_path, csv_file)
                    
                    # 처리 중인 파일 출력
                    print(f"처리 중인 파일(폴더 내): {input_csv}")
                    
                    # CSV 읽기 및 처리
                    process_csv_file(input_csv, output_dir, driver, item)

        elif item.endswith(".csv"):  # 루트에 바로 CSV 파일이 있는 경우 처리
            input_csv = item_path
            
            # 처리 중인 파일 출력
            print(f"처리 중인 파일(루트): {input_csv}")
            
            # CSV 읽기 및 처리
            process_csv_file(input_csv, output_dir, driver, "root")  # "root"는 카테고리 이름 대체용


In [78]:
def process_csv_file(input_csv, output_dir, driver, category):
    """CSV 파일 읽기 및 처리 로직"""
    try:
        df = pd.read_csv(input_csv)
        
        # 열 이름 출력
        print(f"열 이름: {list(df.columns)}")
        
        # 'productId'와 'date' 열 확인
        if "productId" in df.columns and "date" in df.columns:
            print(f"'{input_csv}'에 'productId'와 'date' 열이 있습니다.")
        else:
            print(f"'{input_csv}'에 'productId' 또는 'date' 열이 없습니다.")
            return  # 처리 중단 후 다음 파일로 넘어감

        # 상위 10개 데이터 추출
        top_10_df = df.head(10)
        product_ids = top_10_df["productId"]
        dates = top_10_df["date"]

        # 상세 데이터 수집
        detailed_data = []
        for product_id, date in zip(product_ids, dates):
            details = extract_product_details(driver, product_id, date)
            if details:
                detailed_data.append(details)

        # 결과를 출력 디렉토리에 저장
        output_csv = os.path.join(output_dir, f"{category}_details.csv")
        detailed_df = pd.DataFrame(detailed_data)
        detailed_df.to_csv(output_csv, index=False, encoding="utf-8-sig")
        print(f"{output_csv}에 저장 완료.")

    except Exception as e:
        print(f"파일 처리 중 오류 발생: {e}")

In [93]:
def main():
    input_dir = "ranking_summary_data"
    output_dir = "detailed_data"
    service, driver = start_chromedriver()
    try:
        process_detailed_data(driver, input_dir, output_dir)
    finally:
        end_chromedriver(driver, service)


if __name__ == "__main__":
    main()


Chrome 드라이버 실행.
처리 중인 파일(폴더 내): ranking_summary_data/underwears/20241128-ranking-underwears-summary.csv
열 이름: ['productId', 'brandName', 'productName', 'price', 'discountRate', 'currentlyViewing', 'currentlyBuying', 'ranking', 'trending', 'colors', 'date']
'ranking_summary_data/underwears/20241128-ranking-underwears-summary.csv'에 'productId'와 'date' 열이 있습니다.
접근 중: https://www.musinsa.com/products/4632683
추출된 데이터: {'ProductID': 4632683, 'Date': 20241128, 'Views': 5200, 'Likes': 200000, 'Rating': None, 'TotalSales': None}
접근 중: https://www.musinsa.com/products/4632681
추출된 데이터: {'ProductID': 4632681, 'Date': 20241128, 'Views': 7100, 'Likes': 200000, 'Rating': None, 'TotalSales': None}
접근 중: https://www.musinsa.com/products/4439075
추출된 데이터: {'ProductID': 4439075, 'Date': 20241128, 'Views': 80000, 'Likes': 110000, 'Rating': '4.9', 'TotalSales': 2700}
접근 중: https://www.musinsa.com/products/4252685
추출된 데이터: {'ProductID': 4252685, 'Date': 20241128, 'Views': 60000, 'Likes': 110000, 'Rating': '4